<a href="https://colab.research.google.com/github/Sg0107/tip-calculcator-start/blob/main/digit_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

inputsize = 784
hidden_size = 200
num_classes = 10
num_epochs = 6
batch_size = 200
learning_rate = 0.001
train_dataset = torchvision.datasets.MNIST(root = './data' , train = True, transform = transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root = './data' , train = False, transform = transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

eg = iter(train_loader)
samples, lables = next(eg)
print(samples.shape, lables.shape)

# for i in range(6):
#   plt.subplot(2,3,i+1)
#   plt.imshow(samples[i][0], cmap='Blues')
# # plt.show

class neuralnet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(neuralnet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    return out

model = neuralnet(inputsize, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)

    output = model(images)
    loss = criterion(output,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%100 == 0:
      print(f'{epoch+1}/{num_epochs}, step{i+1}/{n_total_steps}, loss = {loss.item():.4f}')

with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for images, labels in test_loader:
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)
    output = model(images)

    _, predictions = torch.max(output,1)
    n_samples += labels.shape[0]
    n_correct += (predictions == labels).sum().item()

  acc = 100*n_correct/n_samples
  print(f'accuracy = {acc}')
